### Import libraries 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.io as sio
import os
import matplotlib.pyplot as plt 
import itertools
from itertools import groupby, combinations
from all_observables import *
from discrepancy_measures import *
from test_retest_reliability import *

### Load data

In [2]:
file_path = "/.../fmri_state_transition_dynamics/sample_data/" # Please complete the directory when you use the code.
# Use the first session of the first participant 
df = pd.read_csv(file_path+"sample_data_participant1_session1.csv")

### Assign the cluster label to each time point and compute the centroid of each cluster using K-means clustering.

In [3]:
data = np.array(df)
cluster_labels, cluster_centroids = Kmeans(data = data, num_clusters = 4)

### Assign the cluster label to each time point, and calculate GEV and the five observables.


In [4]:
cluster_labels, GEV, observables = lav_observ(data = data,  method = Kmeans, num_clusters = 4)

### Discrepancy measures
- **Dissimilarity between centroid position**
- **Total variation (TV) of coverage time** 
- **TV of frequency** 
- **TV of average lifespan** 
- **Frobenius distance of two transition probability matrices** 

To compare observables between two sessions from the same participant or different participants, the provided code outputs a list "all_results", which is a list of lists. Each inner list contains all observables from a session. The length of the outer list is equal to the number of participants. So, if there are $p$ participants and each of them has $s$ sessions, then "all_results" is a list of length $p$, and each element of "all_results" is a list of length $s$. [NM: Please add a line break (plus either an indent or blank line) here.] The second command line just below outputs "within_participants", which contains the discrepancy measures of observables between two sessions from the same participants. The third command line ouputs "between_participants", which contains the discrepancy measures of observables between two sessions from different participants. Both "within_participants" and "between_participants" are a list. Each element of these lists is an numpy array consisting of the dissimilarity in the centroid position between two sessions, TV of coverage time between the same two sessions, TV of frequency of each state between the two sessions, TV of average lifespan between the two sessions, and the Frobenius distance between the two transition probability matrices. For within-participant comparison, the length of the output list is $p \times {s \choose 2}$, where $p$ is the number of participants for which we want the discrepancy measures, and $s$ is the number of sessions for each sparticipant. For between-participant comparison, the length of the list is $s \times {p \choose 2}$. In the example code, dummy data with $p=2$ participants and $s=2$ sessions per participant is used for illustration.

In [5]:
# all observables from all sessions
all_results = [[lav_observ(data = np.array(pd.read_csv(file_path+"sample_data_participant"+str(p)+"_session"+str(s)+".csv")),  
           method = Kmeans, num_clusters = 4)[2] for s in range(1,3)] for p in range(1,3)]

# Discrepancy in terms of the five observables between sessions within the same participant and between sessions from different participants. 
within_participants = reproducibility(all_results = np.array(all_results), distance = "cosine", 
                                      comparison = "WP")
between_participants = reproducibility(all_results = np.array(all_results), distance = "cosine", 
                                       comparison = "BP")

### Permutation test
We hypothesize that the state-transition dynamics estimated from fMRI data is more consistent between different sessions of the same participant than between different participants. To test this hypothesis, we compare the dissimilarity between two sessions originating from the same participant and the dissimilarity between two sessions originating from different participants. For each observable, we compare the within-participant dissimilarity and between-participant dissimilarity using the normalized distance, denoted by ND, combined with the permutation test. The three outputs are [NM: Can you make the three in the following a bullet-point list? I know how to do it in markdown, but I don't know python notebook grammer...] "ND_empirical" which is a numpy array provides the ND values of five observables computed from empirical data (See Eq. (17) in the article [I will embed the link here][NM: I think it is unnecessary. Same below.]); "ND_permuted" is a (10000,5) numpy array each row provides the ND values of five observables computed after each permutation; "p_value" is the p-value calculated from "ND_empirical" and "ND_permuted" (See section 2.6.3 in the article [I will embed the link here].).

In [6]:
ND_empirical, ND_permuted, p_value = permuted_ND(N = 10000, 
                                                 all_results = np.array(all_results), 
                                                 distance = "cosine")

If you want to get only the ND value for the five observables from empirical data, without running the permutation test, then you can use the following command.

In [7]:
ND_empirical = ND_value(all_results = np.array(all_results), distance = "cosine")